# Day 4

## Tokenizing with code

In [1]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4.1-mini")

tokens = encoding.encode("Hi my name is Ed and I like banoffee pie")

In [2]:
tokens

[12194, 922, 1308, 382, 6117, 326, 357, 1299, 9171, 26458, 5148]

In [3]:
for token_id in tokens:
    token_text = encoding.decode([token_id])
    print(f"{token_id} = {token_text}")

12194 = Hi
922 =  my
1308 =  name
382 =  is
6117 =  Ed
326 =  and
357 =  I
1299 =  like
9171 =  ban
26458 = offee
5148 =  pie


In [4]:
encoding.decode([326])

' and'

# And another topic!

### The Illusion of "memory"

Many of you will know this already. But for those that don't -- this might be an "AHA" moment!

In [5]:
import os
from dotenv import load_dotenv

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if not api_key:
    print("No API key was found - please head over to the troubleshooting notebook in this folder to identify & fix!")
elif not api_key.startswith("sk-proj-"):
    print("An API key was found, but it doesn't start sk-proj-; please check you're using the right key - see troubleshooting notebook")
else:
    print("API key found and looks good so far!")

API key found and looks good so far!


### You should be very comfortable with what the next cell is doing!

_I'm creating a new instance of the OpenAI Python Client library, a lightweight wrapper around making HTTP calls to an endpoint for calling the GPT LLM, or other LLM providers_

In [6]:
from openai import OpenAI

openai = OpenAI()

### A message to OpenAI is a list of dicts

In [8]:
messages = [
    {"role": "system", "content": "あなたはアシスタントです。"},
    {"role": "user", "content": "こんにちは！私はじゅんです！"}
    ]

In [9]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'こんにちは、じゅんさん！どうぞよろしくお願いします。今日はどんなことをお手伝いしましょうか？'

### OK let's now ask a follow-up question

In [12]:
messages = [
    {"role": "system", "content": "あなたは助手です。"},
    {"role": "user", "content": "あなたの名前は何ですか？"}
    ]

In [13]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'私はAIアシスタントです。名前は特にありませんが、お好きな名前で呼んでいただいても大丈夫です。どうぞよろしくお願いします。'

### Wait, wha??

We just told you!

What's going on??

Here's the thing: every call to an LLM is completely STATELESS. It's a totally new call, every single time. As AI engineers, it's OUR JOB to devise techniques to give the impression that the LLM has a "memory".

In [15]:
messages = [
    {"role": "system", "content": "You are a helpful assistant"},
    {"role": "user", "content": "Hi! I'm Ed!"},
    {"role": "assistant", "content": "Hi Ed! How can I assist you today?"},
    {"role": "user", "content": "What's my name?"}
    ]

In [16]:
response = openai.chat.completions.create(model="gpt-4.1-mini", messages=messages)
response.choices[0].message.content

'Your name is Ed! How can I help you today, Ed?'

## To recap
1.LLMへの呼び出しは、毎回ステートレス
つまり、1回1回が完全に新しいリクエストです。

2.私たちは 毎回、これまでの会話をすべて入力として渡している
過去のやり取りを「覚えている」わけではありません。

3.その結果、
LLMが会話の文脈を保持しているように見える
→ これが「記憶があるように感じる」理由です。

4.でも実際には、
それは 錯覚（トリック） にすぎません。
毎回「会話全文」を渡している副作用にすぎない。

5.LLMがやっていることはただ一つで、
与えられたトークン列の続きを、最も確率が高い形で予測しているだけです。

もし入力の中に

「私の名前はEdです」
があって、
その後に
「私の名前は何でしたっけ？」
とあれば、
当然、次に来る答えは「Ed」 になります。

